In [16]:
import tally
import os
import pandas as pd
import json
ds = tally.DataSet(api_key=os.environ.get("TALLY_KEY"))

In [ ]:
project_name = "tally_example_project"

In [17]:
ds.use_quantipy(f"../data/4_{project_name}.json",f"../data/4_{project_name}.csv")

In [18]:
variables = ds.variables()
table_banner=['tabs_gender','tabs_boonage', 'tabs_gen']

In [19]:
table_banner = []
with open('../specs/common_variable_names.json', 'r') as f:
    common_variables = json.load(f)
common_variables = [i.lower() for i in common_variables]
for var in variables['single']:
  if any(match in var for match in common_variables):
    table_banner.append(var)

In [20]:
#questionList = ['q1',
#  'q2',
#  'q3',
#  'q4',
#  'q5'
#]

question_list = variables['single']
question_list = [i for i in question_list if i not in table_banner]

In [22]:
build = tally.Build(name='client A', default_dataset=ds)

####
# Set options sthat apply to whole build/Excel file
####
# set weight
#build.options.set_weight('weight')
# set base outside of main table
build.options.set_base_position('outside')
# brand with font
build.options.set_font('open sans')
# only show percentage results
build.options.set_ci(['c%'])

sheet = build.add_sheet(banner=table_banner)

####
# Set options that apply on a sheet-level
####

# set color and weight of base
sheet.options.set_answer_format(
    'base', 
    {"font_color":"F15A30", "bold":True,'text_wrap': True}
)
sheet.options.set_question_format('percentage', {"bold":True})
sheet.options.set_column_format_for_type('base', 1, {"bold":True})
sheet.options.set_column_format_for_type('percentage', 1, {"bold":True})

####
# Run the calculations and add them to a build
####
for questionlist_index, question in enumerate(question_list):
    if questionlist_index ==0:
        sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})
    else:
        sheet.options.set_base_position('hide')
        sheet.add_table(stub={'x' : question, 'xtotal':True, 'decimals':0})


In [23]:
build.save_excel(f'../outputs/{project_name}.xlsx')

In [24]:
ds.write_quantipy(f"../data/5_{project_name}.json",f"../data/5_{project_name}.csv")